In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql.functions import udf, col, monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, hour, weekofyear, date_format
from pyspark.sql.types import DoubleType, IntegerType, LongType, StringType, StructType, StructField, TimestampType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1651378618735_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
input_data = "s3a://sparkify-bucket4emr/raw/"
output_data = "s3a://sparkify-bucket4emr/processed/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
song_data = os.path.join(input_data, 'song-data', '*', '*', '*', '*.json')
song_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://sparkify-bucket4emr/raw/song-data/*/*/*/*.json'

In [4]:
song_schema = StructType([
        StructField('artist_id', StringType(), False),
        StructField('artist_latitude', StringType(), True),
        StructField('artist_longitude', StringType(), True),
        StructField('artist_location', StringType(), True),
        StructField('artist_name', StringType(), False),
        StructField('song_id', StringType(), False),
        StructField('title', StringType(), False),
        StructField('duration', DoubleType(), True),
        StructField('year', IntegerType(), True)
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
song_df = spark.read.json(song_data, schema = song_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
song_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist_id='ARDR4AC1187FB371A1', artist_latitude=None, artist_longitude=None, artist_location='', artist_name='Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti', song_id='SOBAYLL12A8C138AF9', title='Sono andati? Fingevo di dormire', duration=511.16363, year=0)

In [8]:
log_schema = StructType([
        StructField('artist', StringType(), False),
        StructField('auth', StringType(), True),
        StructField('firstName', StringType(), True),
        StructField('gender', StringType(), True),
        StructField('itemInSession', LongType(), True),
        StructField('lastName', StringType(), True),
        StructField('length', DoubleType(), True),
        StructField('level', StringType(), True),
        StructField('location', StringType(), True),
        StructField('method', StringType(), True),
        StructField('page', StringType(), False),
        StructField('registration', DoubleType(), True),
        StructField('sessionId', LongType(), True),
        StructField('song', StringType(), False),
        StructField('status', LongType(), True),
        StructField('ts', LongType(), False),
        StructField('userAgent', StringType(), True),
        StructField('userId', StringType(), False),
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
log_data = os.path.join(input_data, 'log-data', '*.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
log_df = spark.read.json(log_data, schema = log_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
log_df = log_df.where(log_df['page'] == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# extract columns to create songs table
songs_table = song_df.select('song_id','title','artist_id','year','duration') \
                     .dropDuplicates(['song_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# write songs table to parquet files partitioned by year and artist
songs_table.write.parquet(os.path.join(output_data, 'songs'), mode = 'overwrite', partitionBy = ['year', 'artist_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# extract columns to create artists table
artists_table = song_df.select('artist_id','artist_name','artist_location','artist_latitude','artist_longitude') \
                        .withColumnRenamed('artist_name','artist') \
                        .withColumnRenamed('artist_location','location') \
                        .withColumnRenamed('artist_latitude','latitude') \
                        .withColumnRenamed('artist_longitude','longitude') \
                        .dropDuplicates(['artist_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# write artists table to parquet files
artists_table.write.parquet(os.path.join(output_data, 'artists'), mode = 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# extract columns for users table    
users_table = log_df.select('userId', 'firstName', 'lastName', 'gender', 'level') \
                    .withColumnRenamed('userId', 'user_id') \
                    .withColumnRenamed('firstName', 'first_name') \
                    .withColumnRenamed('lastName', 'last_name') \
                    .dropDuplicates(['user_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# write users table to parquet files
users_table.write.parquet(os.path.join(output_data, 'users'), mode = 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000), TimestampType())
log_df = log_df.withColumn('start_time', get_timestamp(log_df['ts']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# extract columns to create time table
time_table = log_df.select('start_time',
                           hour(col('start_time')).alias('hour'),
                           dayofmonth(col('start_time')).alias('day'),
                           weekofyear(col('start_time')).alias('week'),
                           month(col('start_time')).alias('month'),
                           year(col('start_time')).alias('year'),
                           dayofweek(col('start_time')).alias('weekday')) \
                    .dropDuplicates(['start_time'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# write time table to parquet files partitioned by year and month
time_table.write.parquet(os.path.join(output_data, 'time'), mode = 'overwrite', partitionBy = ['year', 'month'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 45048)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 266, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 254, in authenticate_and_accum_updates
    received_to

In [21]:
# join log and song dataframes to use for songplays table
songplay_df = log_df.join(song_df, (log_df['song'] == song_df['title']) & (log_df['artist'] == song_df['artist_name'])) \
                    .withColumn('songplay_id', monotonically_increasing_id())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# extract columns from joined song and log datasets to create songplays table 
songplays_table = songplay_df.select('songplay_id', 'start_time', col('userId').alias('user_id'), \
                                     'level', 'song_id', 'artist_id', col('sessionId').alias('session_id'), \
                                     'location', col('userAgent').alias('user_agent'), \
                                      year(col('start_time')).alias('year'), \
                                      month(col('start_time')).alias('month')) \
                            .dropDuplicates(['start_time'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.parquet(os.path.join(output_data, 'songplays'), mode = 'overwrite', partitionBy = ['year', 'month'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…